# Excepciones y gestores de contexto
La frase "The best-laid schemes o' mice an' men / Gang aft agley" de Robert Burns debería estar grabado en la mente de todo programador. Aunque nuestro código sea correcto, se producirán errores. Si no los tratamos adecuadamente, pueden hacer que nuestros mejores planes se echen a perder.

Los errores no gestionados pueden hacer que el software se bloquee o se comporte mal. Si tienes suerte, el resultado es un usuario irritado. Si no tiene suerte, su empresa puede acabar perdiendo dinero (un sitio web de comercio electrónico que no para de fallar no suele tener mucho éxito). Por lo tanto, es importante aprender a detectar y gestionar los errores. También es una buena idea cultivar el hábito de pensar siempre en qué errores pueden ocurrir y cómo debería responder tu código cuando ocurran.

Esta sesión trata sobre los errores y cómo tratar con lo inesperado. Aprenderemos sobre excepciones, que son la forma que tiene Python de señalar que ha ocurrido un error u otro evento excepcional. También hablaremos de los gestores de contexto, que proporcionan un mecanismo para encapsular y reutilizar el código de gestión de errores.

En esta sesión vamos a tratar los siguientes temas:
- [Excepciones]()
- [Gestores de contexto]()

# Excepciones


En las sesiones anteriores, ya hemos utilizado algunas excepciones pero no profundizamos sobre ello.Por ejemplo, cuando un iterador se agota, llamar a *next* sobre él lanza una excepción `StopIteration`. Otro tipo de excepción es `IndexError`, que surge cuando intentamos acceder a una lista en una posición que está fuera del rango válido. También hemos visto la excepción `AttributeError` en los ejemplos sin prestar mucho dealle, que surgía cuando intentamos acceder a un atributo en un objeto que no lo tenía, y `KeyError` cuando hicimos lo mismo con una clave y un diccionario.

A veces, aunque una operación o un trozo de código sea correcto, hay condiciones en las que algo puede salir mal. Por ejemplo, si estamos convirtiendo la entrada del usuario de *string* a *int*, el usuario podría escribir accidentalmente una letra en lugar de un dígito, haciendo imposible que convirtamos ese valor en un número. Al dividir números, es posible que no sepamos de antemano si estamos intentando una división por cero. Al abrir un archivo, puede faltar o estar dañado.

Cuando se detecta un error durante la ejecución, se denomina excepción. Las excepciones no son necesariamente letales; de hecho, hemos visto que `StopIteration` está profundamente integrado en los mecanismos generadores e iteradores de Python. Sin embargo, si no tomas las precauciones necesarias, una excepción hará que tu aplicación se rompa. A veces, este es el comportamiento deseado, pero en otros casos, queremos prevenir y controlar problemas como estos. Por ejemplo, podemos alertar al usuario de que el fichero que está intentando abrir está corrupto o que no existe para que pueda arreglarlo o proporcionarle otro fichero, sin necesidad de que la aplicación muera por este problema.

Veamos un ejemplo de algunas excepciones:

In [4]:
gen = (n for n in range(2))
print(f"{next(gen) = }")
print(f"{next(gen) = }")
print(f"{next(gen) = }")

next(gen) = 0
next(gen) = 1


StopIteration: 

In [5]:
print(undefined_name)

NameError: name 'undefined_name' is not defined

In [6]:
mylist = [1, 2, 3, 6]
mylist[5]

IndexError: list index out of range

In [7]:
mydict = {'a': 'A', 'b': 'B'}
mydict['c']

KeyError: 'c'

In [8]:
1 / 0

ZeroDivisionError: division by zero

Como puedes ver, el shell de Python es bastante indulgente. Podemos ver `Traceback`, de modo que tenemos información sobre el error, pero la shell en sí todavía se ejecuta normalmente. Este es un comportamiento especial; un programa normal o un script normalmente saldría inmediatamente si no se hiciera nada para manejar las excepciones. Veamos un ejemplo rápido:

In [9]:
1 + "one"
print("Esta línea nunca será alcanzada")

TypeError: unsupported operand type(s) for +: 'int' and 'str'

Como no hicimos nada para manejar la excepción, Python sale inmediatamente una vez que se produce una excepción (después de imprimir información sobre el error).

## Planteamiento de excepciones
Las excepciones que hemos visto hasta ahora las lanzaba el intérprete de Python cuando detectaba un error. Sin embargo, también puedes lanzar excepciones tú mismo, cuando se produce una situación que tu propio código considera un error. Para lanzar una excepción, utiliza la sentencia `raise`. Por ejemplo:

In [10]:
raise NotImplementedError("I'm afraid I can't do that")

NotImplementedError: I'm afraid I can't do that

Puede utilizar cualquier tipo de excepción que desee, pero es una buena idea elegir el tipo de excepción que mejor describa la condición de error particular que se ha producido. Incluso puedes definir tus propios tipos de excepción. Observe que el argumento que pasamos a la clase Exception se imprime como parte del mensaje de error.

## Definir sus propias excepciones
Como hemos mencionado anteriormente, puedes definir tus propias excepciones personalizadas. Para ello, sólo tienes que definir una clase que herede de cualquier otra clase de excepción. En última instancia, todas las excepciones derivan de `BaseException`; sin embargo, esta clase no está pensada para ser subclasificada directamente, y tus excepciones personalizadas deberían heredar de `Exception` en su lugar. De hecho, casi todas las excepciones incorporadas también heredan de Exception. Las excepciones que no heredan de Exception son para uso interno del intérprete de Python.

## Tracebacks
Los `Tracebacks` que Python imprime pueden parecer inicialmente bastante intimidante, pero es extremadamente útil para entender qué ocurrió para causar la excepción. Echemos un vistazo a un traceback y veamos qué nos puede decir:

In [11]:
def squareroot(number):
    if number < 0:
        raise ValueError("No negative numbers please")
    return number ** .5

def quadratic(a, b, c):
    d = b ** 2 - 4 * a * c
    return ((-b - squareroot(d)) / (2 * a),
            (-b + squareroot(d)) / (2 * a))

quadratic(1, 0, 1) # x**2 + 1 == 0

ValueError: No negative numbers please

En el ejemplo anterior, se define una función llamada $quadratic()$, que utiliza la famosa fórmula cuadrática para encontrar la solución de una ecuación cuadrática. En lugar de utilizar la función $sqrt()$ del módulo `math`, escribimos nuestra propia versión ($squareroot()$), que lanza una excepción si el número es negativo. Cuando llamemos a $quadratic(1, 0, 1)$ para resolver la ecuación $x^2+1=0$, obtendremos un `ValueError`, porque $d$ es negativo.

A menudo resulta útil leer los `tracebacks` de abajo hacia arriba. En la última línea, tenemos el mensaje de error, que nos dice lo que salió mal: `ValueError: No negative numbers please`. Las líneas anteriores nos dicen dónde se produjo la excepción. También podemos ver la secuencia de llamadas a funciones que nos llevaron al punto en el que se produjo la excepción: $squareroot()$ fue llamada por la función $quadratic()$, que fue llamada por el nivel superior del módulo. Como puedes ver, el `traceback` es como un mapa que nos muestra el camino a través del código hasta donde se produjo la excepción. Seguir ese camino y examinar el código en cada función a lo largo del camino es a menudo muy útil cuando quieres entender por qué ocurrió una excepción.

## Tratamiento de excepciones
Para manejar una excepción en Python, se utiliza la sentencia `try`. Cuando se usa, Python estará atento a uno o más tipos diferentes de excepciones (según cómo se lo indiques), y si se producen, te permitirá reaccionar.

La sentencia `try` se compone de la cláusula `try`, que abre la sentencia, seguida de una o más cláusulas `except` que definen qué hacer cuando se captura una excepción. Las cláusulas `except` pueden ir seguidas opcionalmente de una cláusula `else`, que se ejecuta cuando se sale de la cláusula try sin que se haya producido ninguna excepción. Después de las cláusulas `except` y `else` podemos tener una cláusula `finally` (también opcional), cuyo código se ejecuta independientemente de lo que haya ocurrido en las otras cláusulas. La cláusula `finally` se utiliza normalmente para limpiar recursos. También puedes omitir las cláusulas `except` y `else` y tener sólo una cláusula `try` seguida de una cláusula `finally`. Esto es útil si queremos que las excepciones se propaguen y manejen en otro lugar, pero tenemos algún código de limpieza que debe ejecutarse independientemente de si se produce una excepción.

El orden de las cláusulas es importante. Debe ser `try, except, else, finally`. Además, recuerda que `try` debe ir seguida de al menos una cláusula `except` o una cláusula `finally`. Veamos un ejemplo:

In [15]:
def try_syntax(numerator, denominator):
    try:
        print(f'\nIn the try block: {numerator}/{denominator}')
        result = numerator / denominator
    except ZeroDivisionError as zde:
        print(zde)
    else:
        print('The result is:', result)
        return result
    finally:
        print('Exiting')

print(f"{try_syntax(18, 6)}")
print(f"{try_syntax(19, 0)}")


In the try block: 18/6
The result is: 3.0
Exiting
3.0

In the try block: 19/0
division by zero
Exiting
None


Este ejemplo define una sencilla función `try_syntax()`. Realizamos la división de dos números. Estamos preparados para atrapar una excepción `ZeroDivisionError`, que ocurrirá si llamamos a la función con $denominador = 0$. Inicialmente, el código entra en el bloque `try`. Si el denominador no es 0, se calcula el resultado y, tras salir del bloque `try`, se reanuda la ejecución en el bloque `else`. Imprimimos el resultado y lo devolvemos. Echa un vistazo a la salida y verás que justo antes de devolver el resultado, que es el punto de salida de la función, Python ejecuta la cláusula `finally`.

Cuando el $denominador = 0$, las cosas cambian. Nuestro intento de calcular numerador / denominador genera un `ZeroDivisionError`. Como resultado, entramos en el bloque `except` e imprimimos `zde`.El bloque `else` no se ejecuta, porque se lanzó una excepción en el bloque `try`. Antes de devolver (implícitamente) `None`, ejecutamos el bloque `finally`.

Cuando ejecutas un bloque `try`, puede que quieras capturar más de una excepción. Por ejemplo, al llamar a la función `divmod()`, puede obtener un `ZeroDivisionError` si el segundo argumento es 0, o `TypeError` si alguno de los argumentos no es un número. Si desea manejar ambos de la misma manera, puede estructurar su código de la siguiente manera:

In [19]:
values = (1, 2)

try:
    q, r = divmod(*values)
except (ZeroDivisionError, TypeError) as e:
    print(type(e), e)

Este código detectará tanto `ZeroDivisionError` como `TypeError`. Se puede comprobar cambiando $values = (1, 2)$ por $values = (1, 0)$ o $values = ('uno', 2)$, y verá cómo cambia la salida.

Si necesitas manejar diferentes tipos de excepciones de forma diferente, puedes simplemente añadir más cláusulas except, como esta:

In [20]:
try:
    q, r = divmod(*values)
except ZeroDivisionError:
    print("You tried to divide by zero!")
except TypeError as e:
    print(e)

Ten en cuenta que una excepción se maneja en el primer bloque que coincida con esa clase de excepción o cualquiera de sus clases base. Por lo tanto, cuando se apilan varias cláusulas `except` como acabamos de hacer, nos debemos asegurar de poner las excepciones específicas al principio y las genéricas al final. En términos de OOP, los hijos arriba, los abuelos abajo. Además, recuerda que sólo se ejecuta un manejador de excepciones cuando se lanza una excepción.

Python también permite utilizar una cláusula `except` sin especificar ningún tipo de excepción (esto equivale a escribir `except BaseException`). Hacer esto generalmente no es una buena idea, ya que significa que también capturará excepciones que están destinadas al uso interno del intérprete. Estas incluyen las llamadas excepciones de salida del sistema. Éstas son `SystemExit`, que se produce cuando el intérprete sale a través de una llamada a la función `exit()`, y `KeyboardInterrupt`, que se produce cuando el usuario termina la aplicación pulsando *Ctrl + C* (o *Supr* en algunos sistemas).

También se puede lanzar excepciones desde dentro de una cláusula `except`. Por ejemplo, podrías querer reemplazar una excepción incorporada (o una de una librería de terceros) con tu propia excepción personalizada. Esta es una técnica bastante común cuando se escriben bibliotecas, ya que ayuda a proteger a los usuarios de los detalles de implementación de la biblioteca. Veamos un ejemplo:

In [21]:
class NotFoundError(Exception):
    pass

vowels = {'a': 1, 'e': 5, 'i': 9, 'o': 15, 'u': 21}
try:
    pos = vowels['y']
except KeyError as e:
    raise NotFoundError(*e.args)

NotFoundError: y

Por defecto, Python asume que una excepción que se produce dentro de una cláusula `except` es un error inesperado y nos ayuda a imprimir los `tracebacks` de ambas excepciones. Podemos decirle al intérprete que estamos lanzando deliberadamente la nueva excepción usando una sentencia `raise from`:

In [22]:
try:
    pos = vowels['y']
except KeyError as e:
    raise NotFoundError(*e.args) from e

NotFoundError: y

El mensaje de error ha cambiado, pero seguimos teniendo ambas trazas, lo que resulta muy útil para la depuración. Si realmente quisieras suprimir completamente la excepción original, podrías usar `from None` en lugar de `from e`, como se muestra a continuación:

In [23]:
try:
    pos = vowels['y']
except KeyError as e:
    raise NotFoundError(*e.args) from None

NotFoundError: y

También puede utilizar `raise` por sí mismo, sin especificar una nueva excepción, para volver a lanzar la excepción original. Esto a veces es útil si quieres registrar el hecho de que se ha producido una excepción, sin suprimir o reemplazar realmente la excepción.

Programar con excepciones puede ser muy complicado. Podrías ocultar errores inadvertidamente atrapando excepciones que te habrían alertado de su presencia. Vaya a lo seguro teniendo en cuenta estas sencillas directrices:
- Mantenga la cláusula `try` lo más corta posible. Debe contener sólo el código que pueda causar la(s) excepción(es) que desea manejar.
- Haz las cláusulas `except` tan específicas como puedas. Puede ser tentador escribir simplemente except `Exception`, pero si lo hace es casi seguro que acabará capturando excepciones que en realidad no pretendía.
- Utilice pruebas para asegurarse de que su código gestiona correctamente tanto los errores esperados como los inesperados.

Si se siguen estas sugerencias, se minimizará las posibilidades de equivocarse.

## No sólo por los errores
Antes de hablar de los *gestores de contexto*, queremos mostrarte un uso poco convencional de las excepciones, sólo para darte algo que te ayude a ampliar tu visión sobre ellas. Las excepciones se pueden utilizar para algo más que errores:

In [24]:
n = 100
found = False
for a in range(n):
    if found: break
    for b in range(n):
        if found: break
        for c in range(n):
            if 42 * a + 17 * b + c == 5096:
                found = True
                print(a, b, c) # 79 99 95

79 99 95


El código anterior es un modismo bastante común si tratas con números. Tienes que iterar sobre unos cuantos rangos anidados y buscar una combinación particular de $a$, $b$ y $c$ que satisfaga una condición. En este ejemplo, la condición es una ecuación lineal trivial, pero imagina algo mucho mejor que eso. Lo que nos fastidia es tener que comprobar si se ha encontrado la solución al principio de cada bucle, para salir de ellos tan rápido como podamos cuando así sea. La lógica de ruptura interfiere con el resto del código y no nos gusta, por lo que se nos ocurrió una solución diferente para esto. Échale un vistazo y comprueba si puedes adaptarla también a otros casos:

In [25]:
class ExitLoopException(Exception):
    pass

try:
    n = 100
    for a in range(n):
        for b in range(n):
            for c in range(n):
                if 42 * a + 17 * b + c == 5096:
                    raise ExitLoopException(a, b, c)
except ExitLoopException as ele:
    print(ele.args) # (79, 99, 95)

(79, 99, 95)


El código es mucho más elegante. Ahora la lógica de ruptura se maneja completamente con una simple excepción cuyo nombre incluso insinúa su propósito. Tan pronto como se encuentra el resultado, lanzamos `ExitLoopException` con los valores que satisfacen nuestra condición, e inmediatamente se cede el control a la cláusula except que lo maneja. Observa que podemos utilizar el atributo `args` de la excepción para obtener los valores que se pasaron al constructor.

# Gestores de contexto
Cuando trabajamos con recursos externos o estado global, a menudo necesitamos realizar algunos pasos de limpieza, como liberar los recursos o restaurar el estado original, cuando terminamos. No limpiar correctamente podría dar lugar a todo tipo de errores. Por lo tanto, tenemos que asegurarnos de que nuestro código de limpieza se ejecutará incluso si se produce una excepción. Podríamos usar sentencias `try/finally`, pero esto no siempre es conveniente y podría resultar en mucha repetición, ya que a menudo tenemos que realizar pasos de limpieza similares cada vez que trabajamos con un tipo particular de recurso. Los gestores de contexto resuelven este problema creando un contexto de ejecución en el que podemos trabajar con un recurso o estado modificado y realizar automáticamente cualquier limpieza necesaria cuando salimos de ese contexto, incluso si se ha producido una excepción.

Un ejemplo de estado global que podemos querer modificar temporalmente es la precisión de los cálculos decimales. Por ejemplo, supongamos que necesitamos realizar un cálculo particular con una precisión específica, pero queremos mantener la precisión por defecto para el resto de nuestros cálculos. Podríamos hacer algo como lo siguiente:

In [27]:
from decimal import Context, Decimal, getcontext, setcontext

one = Decimal("1")
three = Decimal("3")

orig_ctx = getcontext()
ctx = Context(prec=5)
setcontext(ctx)
print(f"{ctx = }")
print(f"{one / three = }")
setcontext(orig_ctx)
print(f"{one / three = }")

ctx = Context(prec=5, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])
one / three = Decimal('0.33333')
one / three = Decimal('0.3333333333333333333333333333')


Observe que almacenamos el contexto actual, establecemos un nuevo contexto (con una precisión modificada), realizamos nuestro cálculo y, finalmente, restauramos el contexto original.

Esto parece correcto, pero ¿qué pasaría si se produjera una excepción antes de que pudiéramos restaurar el contexto original? Nos quedaríamos atascados con la precisión incorrecta y los resultados de todos los cálculos posteriores serían incorrectos. Podemos solucionarlo utilizando una sentencia `try/finally`:

In [28]:
orig_ctx = getcontext()
ctx = Context(prec=5)
setcontext(ctx)
try:
    print(f"{ctx = }")
    print(f"{one / three = }")
finally:
    setcontext(orig_ctx)
    print(f"{one / three = }")

ctx = Context(prec=5, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])
one / three = Decimal('0.33333')
one / three = Decimal('0.3333333333333333333333333333')


Eso es mucho más seguro. Ahora podemos estar seguros de que, independientemente de lo que ocurra en ese bloque `try`, siempre restauraremos el contexto original. Sin embargo, no es muy conveniente tener que seguir escribiendo `try/finally` así. Aquí es donde los gestores de contexto vienen al rescate. El módulo decimal proporciona el gestor de contexto `localcontext`, que se encarga de establecer y restaurar el contexto por nosotros:

In [29]:
from decimal import localcontext
with localcontext(Context(prec=5)) as ctx:
    print(f"{ctx = }")
    print(f"{one / three = }")
print(f"{one / three = }")

ctx = Context(prec=5, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])
one / three = Decimal('0.33333')
one / three = Decimal('0.3333333333333333333333333333')


Es mucho más fácil de leer (y de escribir). La sentencia `with` se utiliza para entrar en un contexto de ejecución definido por un gestor de contexto. Al salir del bloque de código delimitado por la sentencia `with`, cualquier operación de limpieza definida por el gestor de contexto (en este caso, restaurar el contexto decimal) se ejecuta automáticamente.

También es posible combinar varios gestores de contexto en una sentencia with. Esto resulta muy útil en situaciones en las que es necesario trabajar con varios recursos al mismo tiempo:

In [31]:
with localcontext(Context(prec=5)), open("out.txt", "w") as out_f:
    out_f.write(f"{one} / {three} = {one / three}\n")

Aquí, entramos en un contexto local y abrimos un fichero (que actúa como gestor de contexto) en una sentencia `with`. Realizamos un cálculo y escribimos el resultado en el fichero. Cuando terminamos, el fichero se cierra automáticamente y se restaura el contexto decimal por defecto. No te preocupes demasiado por los detalles de trabajar con ficheros por ahora; aprenderemos todo sobre eso más adelante.

Aparte de los contextos decimales y los ficheros, muchos otros objetos de la biblioteca estándar de Python pueden usarse como gestores de contexto. Por ejemplo:
- Los `objetos Socket`, que implementan una interfaz de red de bajo nivel, pueden usarse como gestores de contexto para cerrar automáticamente conexiones de red. 
- Las `clases de bloqueo` utilizadas para la sincronización en la programación concurrente utilizan el protocolo de gestor de contexto para liberar automáticamente los bloqueos.

A continuación seguiremos mostrando cómo se puede implementar tus propios gestores de contexto.

## Gestores de contexto basados en clases
Los gestores de contexto funcionan mediante dos métodos mágicos: `__enter__()` se llama justo antes de entrar en el cuerpo de la sentencia `with` y `__exit__()` se llama al salir del cuerpo de la sentencia `with`. Esto significa que puedes crear fácilmente tu propio gestor de contexto simplemente escribiendo una clase que implemente estos métodos:

In [32]:
class MyContextManager:
    def __init__(self):
        print("MyContextManager init", id(self))

    def __enter__(self):
        print("Entering 'with' context")
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        print(f"{exc_type=} {exc_val=} {exc_tb=}")
        print("Exiting 'with' context")
        return True

En el código anterior se ha definido una clase de gestor de contexto muy simple llamada `MyContextManager`. En ella, existen algunas cosas interesantes a tener en cuenta. Se puede observar que nuestro método `__enter__()` devuelve `self`. Esto es bastante común, pero no es obligatorio: puedes devolver lo que quieras de `__enter__()`, incluso `None`. El valor de retorno del método `__enter__()` se asignará a la variable nombrada en la cláusula `as` de la sentencia `with`. Otra cosa que se puede observar son los parámetros `exc_type`, `exc_val` y `exc_tb` de la función `__exit__()`. Si se lanza una excepción dentro del cuerpo de la sentencia `with`, el intérprete pasará el tipo, valor y traza de la excepción como argumentos a través de estos parámetros. Si no se produce ninguna excepción, los tres argumentos serán `None`.

Observe también que nuestro método `__exit__()` devuelve `True`. Esto hará que se suprima cualquier excepción lanzada dentro del cuerpo de la sentencia `with` (como si la hubiéramos manejado en una sentencia `try/except`). Si por el contrario hubiéramos devuelto `False`, tal excepción continuaría propagándose después de que nuestro método `__exit__()` se hubiera ejecutado. La capacidad de suprimir excepciones significa que un gestor de contexto puede ser utilizado como un manejador de excepciones. El beneficio de esto es que podemos escribir nuestra lógica de manejo de excepciones una vez y reutilizarla donde la necesitemos. Esta es sólo una forma más en la que Python nos ayuda a aplicar el principio *DRY* a nuestro código.

Veamos nuestro gestor de contexto en acción:

In [33]:
ctx_mgr = MyContextManager()
print("About to enter 'with' context")

with ctx_mgr as mgr:
    print("Inside 'with' context")
    print(id(mgr))
    raise Exception("Exception inside 'with' context")
    print("This line will never be reached")
print("After 'with' context")

MyContextManager init 2078242598176
About to enter 'with' context
Entering 'with' context
Inside 'with' context
2078242598176
exc_type=<class 'Exception'> exc_val=Exception("Exception inside 'with' context") exc_tb=<traceback object at 0x000001E3E15BDD80>
Exiting 'with' context
After 'with' context


Aquí, hemos instanciado nuestro gestor de contexto en una sentencia separada, antes de la sentencia `with`. Hicimos esto para que sea más fácil para ver lo que está sucediendo; sin embargo, es mucho más común que esos pasos se combinen como con `MyContextManager()` as `mgr`.

En la salida se muestra algunos `IDs` para ayudar a verificar que el objeto asignado a `mgr` es realmente el mismo objeto que devolvimos de `__enter__()`.

## Gestores de contexto basados en generadores
Si estás implementando una clase que representa algún recurso que necesita ser adquirido y liberado, tiene sentido implementar esa clase como un gestor de contexto. A veces, sin embargo, queremos implementar el comportamiento del gestor de contexto, pero no tenemos una clase a la que tenga sentido adjuntar ese comportamiento. Por ejemplo, puede que sólo queramos utilizar un gestor de contexto para reutilizar alguna lógica de gestión de errores. En tales situaciones, sería bastante tedioso tener que escribir una clase adicional sólo para implementar el comportamiento deseado del gestor de contexto. Afortunadamente para nosotros, Python tiene una solución.

El módulo `contextlib` de la biblioteca estándar proporciona un práctico decorador `contextmanager` que toma una función generadora y la convierte en un gestor de contexto. Entre bastidores, el decorador envuelve el generador en un objeto gestor de contexto. El método `__enter__()` de este objeto inicia el generador y devuelve lo que el generador produzca. Si ocurre una excepción dentro del cuerpo de la sentencia `with`, el método `__exit__()` pasa la excepción al generador (usando el método throw del generador). En caso contrario, `__exit__()` simplemente llama a next en el generador. Tenga en cuenta que el generador sólo debe producir una vez; se producirá un `RuntimeError` si el generador produce una segunda vez. Traduzcamos nuestro ejemplo anterior a un gestor de contexto basado en un generador:

In [34]:
from contextlib import contextmanager

@contextmanager
def my_context_manager():
    print("Entering 'with' context")
    val = object()
    print(id(val))
    
    try:
        yield val
    except Exception as e:
        print(f"{type(e)=} {e=} {e.__traceback__=}")
    finally:
        print("Exiting 'with' context")

print("About to enter 'with' context")
with my_context_manager() as val:
    print("Inside 'with' context")
    print(id(val))
    raise Exception("Exception inside 'with' context")
    print("This line will never be reached")

print("After 'with' context")

About to enter 'with' context
Entering 'with' context
2078246971936
Inside 'with' context
2078246971936
type(e)=<class 'Exception'> e=Exception("Exception inside 'with' context") e.__traceback__=<traceback object at 0x000001E3E15C43C0>
Exiting 'with' context
After 'with' context


La mayoría de los generadores de gestores de contexto tienen una estructura similar a `my_context_manager()` en este ejemplo. Tienen algo de código de configuración, seguido de una sentencia `yield` dentro de una sentencia `try`. Aquí, hemos cedido un objeto arbitrario, para que puedas ver que el mismo objeto está disponible a través de la cláusula `as` de la sentencia `with`. También es bastante común tener sólo un `yield` sin valor (en cuyo caso se cede `None`). 

En estos casos, la cláusula as de la sentencia with suele omitirse. Una característica muy útil de los gestores de contexto basados en generadores es que también pueden utilizarse como decoradores de funciones. Esto significa que si todo el cuerpo de una función necesita estar dentro de un contexto de sentencia `with`, puedes ahorrarte un nivel de indentación y simplemente decorar la función en su lugar.